In [ ]:
!kaggle datasets download -d andrewmvd/car-plate-detection


Dataset URL: https://www.kaggle.com/datasets/andrewmvd/car-plate-detection
License(s): CC0-1.0
 99% 202M/203M [00:09<00:00, 24.0MB/s]
100% 203M/203M [00:09<00:00, 21.4MB/s]


In [ ]:
!unzip -q car-plate-detection.zip -d /content/car_plate_dataset


In [ ]:
import os
import cv2
import shutil
import numpy as np
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt
import xml.etree.ElementTree as xet
from sklearn.model_selection import train_test_split
import torch
print(f'{torch.cuda.is_available() =  }')
print(f'{torch.cuda.device_count() =}')

torch.cuda.is_available() =  True
torch.cuda.device_count() =1


In [ ]:
! pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.6/914.6 kB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 877.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninsta

In [ ]:
!pip install -U ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 61.4 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1


In [ ]:
dataset_path='/content/car_plate_dataset'

In [ ]:
import re

def the_number_in_the_string(filename):
    match=re.search(r'(\d+)',filename)
    if match:
        return int(match.group(0))
    else:
        return 0

In [ ]:
print(the_number_in_the_string('file123.txt'))

123


In [ ]:
labels_dict=dict(
    img_path=[],
    xmin=[],
    xmax=[],
    ymin=[],
    ymax=[],
    img_w=[],
    img_h=[]
)

In [ ]:
labels_dict

{'img_path': [],
 'xmin': [],
 'xmax': [],
 'ymin': [],
 'ymax': [],
 'img_w': [],
 'img_h': []}

In [ ]:
xml_files=glob(f'{dataset_path}/annotations/*.xml')
xml_files

['/content/car_plate_dataset/annotations/Cars217.xml',
 '/content/car_plate_dataset/annotations/Cars336.xml',
 '/content/car_plate_dataset/annotations/Cars233.xml',
 '/content/car_plate_dataset/annotations/Cars106.xml',
 '/content/car_plate_dataset/annotations/Cars420.xml',
 '/content/car_plate_dataset/annotations/Cars75.xml',
 '/content/car_plate_dataset/annotations/Cars159.xml',
 '/content/car_plate_dataset/annotations/Cars129.xml',
 '/content/car_plate_dataset/annotations/Cars20.xml',
 '/content/car_plate_dataset/annotations/Cars147.xml',
 '/content/car_plate_dataset/annotations/Cars273.xml',
 '/content/car_plate_dataset/annotations/Cars169.xml',
 '/content/car_plate_dataset/annotations/Cars353.xml',
 '/content/car_plate_dataset/annotations/Cars384.xml',
 '/content/car_plate_dataset/annotations/Cars261.xml',
 '/content/car_plate_dataset/annotations/Cars418.xml',
 '/content/car_plate_dataset/annotations/Cars392.xml',
 '/content/car_plate_dataset/annotations/Cars314.xml',
 '/content/c

In [ ]:
for filename in sorted(xml_files,key=the_number_in_the_string):
    info=xet.parse(filename)
    root=info.getroot()

    member_object=root.find('object')
    labels_info=member_object.find('bndbox')

    xmin=int(labels_info.find('xmin').text)
    xmax=int(labels_info.find('xmax').text)
    ymin=int(labels_info.find('ymin').text)
    ymax=int(labels_info.find('ymax').text)

    img_name=root.find('filename').text
    img_path=os.path.join(dataset_path,'images',img_name)

    height,width,_=cv2.imread(img_path).shape

    labels_dict['img_path'].append(img_path)
    labels_dict['xmin'].append(xmin)
    labels_dict['xmax'].append(xmax)
    labels_dict['ymin'].append(ymin)
    labels_dict['ymax'].append(ymax)
    labels_dict['img_w'].append(width)
    labels_dict['img_h'].append(height)



In [ ]:
 alldata=pd.DataFrame(labels_dict)
alldata

,img_path,xmin,xmax,ymin,ymax,img_w,img_h
0,/content/car_plate_dataset/images/Cars0.png,226,419,125,173,500,268
1,/content/car_plate_dataset/images/Cars1.png,134,262,128,160,400,248
2,/content/car_plate_dataset/images/Cars2.png,229,270,176,193,400,400
3,/content/car_plate_dataset/images/Cars3.png,142,261,128,157,400,225
4,/content/car_plate_dataset/images/Cars4.png,156,503,82,253,590,350
...,...,...,...,...,...,...,...
428,/content/car_plate_dataset/images/Cars428.png,142,258,128,157,400,225
429,/content/car_plate_dataset/images/Cars429.png,86,208,166,195,301,400
430,/content/car_plate_dataset/images/Cars430.png,38,116,159,197,400,225
431,/content/car_plate_dataset/images/Cars431.png,55,343,82,147,400,192


In [ ]:
train,test=train_test_split(alldata,test_size=1/10,random_state=42)
train,val=train_test_split(train,train_size=8/9,random_state=42)
print(f'''
len(train)={len(train)}
len(val)={len(val)}
len(test)={len(test)}''')


len(train)=345
len(val)=44
len(test)=44


In [ ]:
def make_split_folder_in_yolo_format(split_name, split_df):
    labels_path = os.path.join('datasets', 'cars_license_plate_new', split_name, 'labels')
    images_path = os.path.join('datasets', 'cars_license_plate_new', split_name, 'images')

    os.makedirs(labels_path, exist_ok=True)
    os.makedirs(images_path, exist_ok=True)

    for _, row in split_df.iterrows():
        img_name, img_extension = os.path.splitext(os.path.basename(row['img_path']))

        # Ensure bounding box is valid
        xmin, xmax = min(row['xmin'], row['xmax']), max(row['xmin'], row['xmax'])
        ymin, ymax = min(row['ymin'], row['ymax']), max(row['ymin'], row['ymax'])

        # Compute YOLO format values
        x_center = (xmin + xmax) / 2 / row['img_w']
        y_center = (ymin + ymax) / 2 / row['img_h']
        width = (xmax - xmin) / row['img_w']
        height = (ymax - ymin) / row['img_h']

        # Ensure values are within the valid range [0, 1]
        x_center = max(0, min(1, x_center))
        y_center = max(0, min(1, y_center))
        width = max(0, min(1, width))
        height = max(0, min(1, height))

        # Skip if bounding box is too small
        if width <= 0 or height <= 0:
            print(f"Skipping {img_name}: Invalid width/height")
            continue

        label_path = os.path.join(labels_path, f'{img_name}.txt')
        with open(label_path, 'w') as file:
            file.write(f"0 {x_center:.4f} {y_center:.4f} {width:.4f} {height:.4f}\n")

        shutil.copy(row['img_path'], os.path.join(images_path, img_name + img_extension))

    print(f"Created '{images_path}' and '{labels_path}'")

In [ ]:
make_split_folder_in_yolo_format('train', train)
make_split_folder_in_yolo_format('val', val)
make_split_folder_in_yolo_format('test', test)

Created 'datasets/cars_license_plate_new/train/images' and 'datasets/cars_license_plate_new/train/labels'
Created 'datasets/cars_license_plate_new/val/images' and 'datasets/cars_license_plate_new/val/labels'
Created 'datasets/cars_license_plate_new/test/images' and 'datasets/cars_license_plate_new/test/labels'


In [ ]:
datasets_yaml = """
path: cars_license_plate_new
train: train/images
val: val/images
test: test/images

nc: 1
names: ["license_plate"]
"""

with open("datasets.yaml", "w") as file:
    file.write(datasets_yaml)


In [ ]:
from ultralytics import YOLO


In [ ]:
model=YOLO('yolov8n.pt')

In [ ]:
model.train(
    data='datasets.yaml',
    epochs=100,
    batch=16,
    device='cuda',
    imgsz=320,
    cache=True

)

Ultralytics 8.3.73 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=datasets.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=320, save=True, save_period=-1, cache=True, device=cuda, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_

train: Scanning /content/datasets/cars_license_plate_new/train/labels... 345 images, 0 backgrounds, 0 corrupt: 100%|██████████| 345/345 [00:00<00:00, 523.10it/s]

train: New cache created: /content/datasets/cars_license_plate_new/train/labels.cache


WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


train: Caching images (0.1GB RAM): 100%|██████████| 345/345 [00:04<00:00, 71.16it/s]


...

                   all         44         44      0.929      0.818      0.896      0.545



100 epochs completed in 0.133 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 6.2MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics 8.3.73 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  5.27it/s]


                   all         44         44      0.918      0.909      0.929      0.567
Speed: 0.1ms preprocess, 1.3ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to runs/detect/train2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7b1605191310>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [ ]:
import os
from glob import glob  # Import the glob module

log_dir = max(glob('runs/detect/train*'), key=the_number_in_the_string)
print(log_dir)


runs/detect/train2


In [ ]:
results=pd.read_csv(os.path.join(log_dir,'results.csv'))
results

,epoch,time,train/box_loss,train/cls_loss,train/dfl_loss,metrics/precision(B),metrics/recall(B),metrics/mAP50(B),metrics/mAP50-95(B),val/box_loss,val/cls_loss,val/dfl_loss,lr/pg0,lr/pg1,lr/pg2
0,1,10.2218,1.75705,3.01153,1.36535,0.00394,0.93182,0.24864,0.14178,1.42223,2.98659,1.08809,0.000420,0.000420,0.000420
1,2,14.4314,1.47705,1.44231,1.14870,0.48524,0.17176,0.22462,0.11532,1.69170,2.81313,1.22793,0.000851,0.000851,0.000851
2,3,20.7868,1.55989,1.47003,1.22451,1.00000,0.15124,0.76393,0.34829,1.74600,2.51282,1.51320,0.001274,0.001274,0.001274
3,4,25.2662,1.55861,1.31739,1.22627,0.82710,0.43499,0.63733,0.30677,1.76844,2.10190,1.39772,0.001688,0.001688,0.001688
4,5,28.5687,1.51811,1.31300,1.19826,0.60129,0.45455,0.54707,0.27137,1.71077,1.67669,1.34387,0.001921,0.001921,0.001921
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,461.7910,0.93244,0.49710,0.99437,0.92174,0.81818,0.89064,0.54555,1.42378,0.64560,1.25546,0.000119,0.000119,0.000119
96,97,465.6970,0.92510,0.48277,0.97843,0.93947,0.84091,0.91747,0.55592,1.41747,0.63597,1.25437,0.000099,0.000099,0.000099
97,98,471.2780,0.95262,0.50147,0.99358,0.91849,0.81818,0.89612,0.54078,1.39863,0.61737,1.23560,0.000079,0.000079,0.000079
98,99,475.1670,0.93371,0.47768,0.98855,0.93932,0.84091,0.92291,0.54868,1.40260,0.61796,1.22826,0.000060,0.000060,0.000060


In [ ]:
results.columns=results.columns.str.strip()
results.columns

NameError: name 'results' is not defined

In [ ]:
epochs=results.index+1
print(epochs)

In [ ]:
map_05=results['metrics/mAP50(B)']
map_05095=results['metrics/mAP50-95(B)']
plt.figure(figsize=(10,7))
plt.plot(epochs,map_05,label='mAP@0.5')
plt.plot(epochs,map_05095,label='mAP@0.5:0.95')
plt.xlabel('Epochs')
plt.ylabel('mAP')
plt.title('YOLOv8 Performance')
plt.legend()
plt.show()

In [ ]:
model.save('best_license_plate_model.pt')

In [ ]:
results=model.predict('/content/car_plate_dataset/images/Cars100.png')

In [ ]:
results

In [ ]:
os.litsdir('runs/detect/')

In [ ]:
image=cv2.imread('/content/car_plate_dataset/images/Cars100.png')
image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
plt.figure(figsize=(10,10))
plt.imshow(image)
plt.show()

In [ ]:
for result in results:
  for box in result.boxes:
    print(box)
    x1,y1,x2,y2=map(int,box,xyxy[0])
    confidencej=box.conf[0]
    cv2.rectangle(image,(x1,y1),(x2,y2),(0,255,0),2)
    cv2.putText(image,F'{confidence:.2f}',(x1,y1-10),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),2)
    break
  break
plt.imshow(image)
plt.axis('off')
plt.show()
